## Ingestion del archivo "country.json"

#### `Paso 0 - Actualización de variables y funciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze

In [0]:
#display(spark.read.option("header", True).json("/mnt/moviehistoryadilmor/bronze/country.json"))

In [0]:
#Definindo o schema na variavel counties_schema
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read \
               .schema(countries_schema) \
               .json(f"{bronze_folder_path}/{v_file_date}/country.json")

In [0]:
#countries_df.printSchema()
#display(countries_df)

#### Paso 2 - Eliminar las columnas no "deseadas" del Dataframe

In [0]:
#1ª forma
#countries_dropped_df = countries_df.drop("countryIsoCode")

#2ª forma
#countries_dropped_df = countries_df.drop(countries_df["countryIsoCode"])

#3ª forma
from pyspark.sql.functions import col
countries_dropped_df = countries_df.drop(col("countryIsoCode"))


#### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

countries_final_df = add_ingestion_date(countries_dropped_df) \
                    .withColumnRenamed("countryId", "country_id") \
                    .withColumnRenamed("countryName", "country_name") \
                    .withColumn("enviroment", lit("production")) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
#display(countries_final_df)

#### Paso 5 - Escribir datos en el datalake en formato parquet

In [0]:
#countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/countries/")
#countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.countries")
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/countries

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}countries"))
#spark.sql("select * from movie_silver.countries").display()

In [0]:
dbutils.notebook.exit("Success")